In [1]:
import os
import pandas as pd
import re
from datetime import datetime
import yaml
from scripts.clean_data import * #Local Module

In [2]:
#Iterate over the data in the folder and store the cleaned data as a dataframe in the 'dataframes' dictionary.
folder_path = 'data\\1_raw'

# Initialize a dictionary to hold the DataFrames
dataframes = {}

# List all files in the folder
files = os.listdir(folder_path)

# Regular expression to match the file names and extract the year
pattern = r'texas_border_report_(\d{4}).xlsx'

for file_name in files:
    match = re.match(pattern, file_name)
    if match:
        year = match.group(1)
        file_path = os.path.join(folder_path, file_name)
        
        print(f"Loading file: {file_name}")
        
        # Load the Excel file into a DataFrame
        df = pd.read_excel(
            file_path, 
            sheet_name='By Agency', 
            skiprows=1, 
            skipfooter=6,
            dtype={'NIBRS Start Date': 'object'}
        )
        
        #Date is formated as MMM-YY in excel, but the underlying data has the day. Going to stick with the MMM-YY formating for now
        df['NIBRS Start Date'] = pd.to_datetime(df['NIBRS Start Date'], format='%b-%y')
        df['Population'] = df['Population'].str.replace(',', '').astype(int)
        df.drop(columns=['Total'], inplace=True)
        
        superscript_removal_map = str.maketrans('', '', '¹²³⁴⁵⁶⁷⁸⁹⁰')
        df.columns = [col.lower().replace('  ', '_').replace(' ', '_').replace(',','').translate(superscript_removal_map) for col in df.columns]

        # Store the DataFrame in the dictionary with a dynamic key
        dataframes[f'report_{year}'] = df


Loading file: texas_border_report_2017.xlsx
Loading file: texas_border_report_2018.xlsx
Loading file: texas_border_report_2019.xlsx
Loading file: texas_border_report_2020.xlsx
Loading file: texas_border_report_2021.xlsx
Loading file: texas_border_report_2022.xlsx
Loading file: texas_border_report_2023.xlsx


In [3]:
#Create a reference report and define it as the most recently completed report as a reference
reference_report, old_reports = extract_reference_df('report_2023', dataframes)

#Checking to make sure all the shapes of the dataframes are the same
check_shape(reference_report, old_reports)

#Generate list of columns in the current report
current_report_columns = list(reference_report.columns)

#Check the columns
check_columns(current_report_columns, old_reports)

All dataframes have the same shape.
The columns in the old reports are the same as the columns in the reference report


In [4]:
#Check to make sure all of the agency names are consistent across all of the reports
agencies_names = check_column_values('agency_name', reference_report, old_reports)

Column values in the reference dataframe match the values in the 'report_2017' dataframe.
Column values in the reference dataframe match the values in the 'report_2018' dataframe.
Column values in the reference dataframe match the values in the 'report_2019' dataframe.
Column values in the reference dataframe match the values in the 'report_2020' dataframe.
Column values in the reference dataframe match the values in the 'report_2021' dataframe.
Column values in the reference dataframe match the values in the 'report_2022' dataframe.


In [5]:
#Create a dataframe with all of the data from the reports

#Adding the current report dataframe back into the dictionary
dataframes['report_2023'] = reference_report

#Extract and combine the dataframes 
for key, df in dataframes.items():
    year = int(key.split('_')[1])  # Extract the year from the key
    date_str = f'12/31/{year}' # Create a string representing '12/31/YYYY' for each row
    
    # Convert the string to datetime and assign to the new column 'report_year'
    df['report_year'] = pd.to_datetime(date_str, format='%m/%d/%Y')
    
    globals()[key] = df  # Update the global variable with the modified DataFrame


# Combine all DataFrames into a single DataFrame
border_report = pd.concat(dataframes.values(), ignore_index=True)

#Check to make sure all of the years are in the report
border_report['report_year'].dt.strftime('%Y').unique()

array(['2017', '2018', '2019', '2020', '2021', '2022', '2023'],
      dtype=object)

In [6]:
border_report.head()

,agency_name,county,population,nibrs_start_date,murder_and_nonnegligent_manslaughter,negligent_manslaughter,rape,robbery,assault,burglary,larceny_theft,motor_vehicle_theft,arson,human_trafficking_commercial_sex_acts,human_trafficking_involuntary_servitude,months_reported,report_year
0,ALAMO PD,Hidalgo County,19351,2018-08-01,0,0,0,0,0,0,0,0,0,0,0,0,2017-12-31
1,ALPINE PD,Brewster County,5983,2020-04-01,0,0,0,0,0,0,0,0,0,0,0,0,2017-12-31
2,ALTON PD,Hidalgo County,17432,2019-09-01,0,0,0,0,0,0,0,0,0,0,0,0,2017-12-31
3,ANTHONY PD,El Paso County,5681,2020-12-01,0,0,0,0,0,0,0,0,0,0,0,0,2017-12-31
4,BREWSTER CO SO,Brewster County,3196,2015-01-01,1,0,2,0,19,4,9,3,1,0,0,11,2017-12-31


In [7]:
#Store resulting dataframe into 2_staging
border_report.to_csv('data/2_staging/stg_texas_border_report.csv', index=False)